In [37]:
import pandas as pd
import numpy as np
from sklearn.cross_validation import train_test_split

In [2]:
train_path = '/Users/ngochuyenluu/Downloads/BNP/train.csv'
test_path = '/Users/ngochuyenluu/Downloads/BNP/test.csv'

In [3]:
def dense_to_one_hot(labels_dense, num_classes=10):
  """
  Convert class labels from scalars to one-hot vectors.
  http://stackoverflow.com/questions/33681517/tensorflow-one-hot-encoder
  """
  labels_dense = np.array(labels_dense)
  num_labels = labels_dense.shape[0]
  index_offset = np.arange(num_labels) * num_classes
  labels_one_hot = np.zeros((num_labels, num_classes))
  labels_one_hot.flat[index_offset + labels_dense.ravel()] = 1
  return labels_one_hot

In [4]:
def process_data(path, is_test_set=False):

    df = pd.read_csv(path)
    
    if is_test_set:
        df = df.interpolate()

    # Ignoring categorical input
    df = df.drop('v3', axis=1)
    df = df.drop('v22', axis=1)
    df = df.drop('v24', axis=1)
    df = df.drop('v30', axis=1)
    df = df.drop('v31', axis=1)
    df = df.drop('v47', axis=1)
    df = df.drop('v52', axis=1)
    df = df.drop('v56', axis=1)
    df = df.drop('v66', axis=1)
    df = df.drop('v71', axis=1)
    df = df.drop('v74', axis=1)
    df = df.drop('v75', axis=1)
    df = df.drop('v79', axis=1)
    df = df.drop('v91', axis=1)
    df = df.drop('v107', axis=1)
    df = df.drop('v110', axis=1)
    df = df.drop('v112', axis=1)
    df = df.drop('v113', axis=1)
    df = df.drop('v125', axis=1)
    
    if not is_test_set:
        # Ignore datapoints with missing values
        df = df.dropna()
        
        labels = df['target'].values.tolist()
        labels = dense_to_one_hot(labels, 2)
        df = df.drop('ID', axis=1)
        df = df.drop('target', axis=1)
        
        # Normalize the data
        df_norm = (df - df.mean()) / (df.max() - df.min())
        data = df_norm.values.tolist()

        X_train, X_test, y_train, y_test = train_test_split(data, labels,
                                                test_size=0.15, random_state=42)
        return X_train, X_test, y_train, y_test

    else:
        ids = df['ID'].values.tolist()
        df = df.drop('ID', axis=1)
        df_norm = (df - df.mean()) / (df.max() - df.min())
        data = df_norm.values.tolist()
        return data, ids

In [5]:
X_train, X_test, y_train, y_test = process_data(train_path)

In [38]:
y_test

array([[ 0.,  1.],
       [ 0.,  1.],
       [ 1.,  0.],
       ..., 
       [ 1.,  0.],
       [ 0.,  1.],
       [ 0.,  1.]])

In [6]:
np.random.seed(1337)  # for reproducibility
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout, advanced_activations
from keras.callbacks import EarlyStopping, History, ModelCheckpoint
from keras import regularizers
from keras import optimizers

Using TensorFlow backend.


In [7]:
print(len(X_train[0]))

112


In [46]:
l1= Sequential()
l1.add(Dense(150, input_dim=112,activation='tanh'))
l1.add(Dense(50, activation='relu'))
l1.add(Dense(20, activation='relu'))
l1.add(Dense(2,activation='softmax'))
optimizer = optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)
l1.compile(loss='categorical_crossentropy',optimizer=optimizer,metrics=['mse','accuracy'])
l1.fit(X_train, y_train, validation_data=(X_test,y_test),batch_size=100, epochs=100, verbose=1)

Train on 53176 samples, validate on 9385 samples
Epoch 1/100
53176/53176 [==============================] - 2s - loss: 0.5284 - mean_squared_error: 0.1756 - acc: 0.7526 - val_loss: 0.5226 - val_mean_squared_error: 0.1731 - val_acc: 0.7535
Epoch 2/100
53176/53176 [==============================] - 2s - loss: 0.5157 - mean_squared_error: 0.1707 - acc: 0.7544 - val_loss: 0.5180 - val_mean_squared_error: 0.1716 - val_acc: 0.7530
Epoch 3/100
53176/53176 [==============================] - 2s - loss: 0.5138 - mean_squared_error: 0.1699 - acc: 0.7547 - val_loss: 0.5207 - val_mean_squared_error: 0.1730 - val_acc: 0.7492
Epoch 4/100
53176/53176 [==============================] - 2s - loss: 0.5109 - mean_squared_error: 0.1689 - acc: 0.7571 - val_loss: 0.5166 - val_mean_squared_error: 0.1713 - val_acc: 0.7560
Epoch 5/100
53176/53176 [==============================] - 2s - loss: 0.5099 - mean_squared_error: 0.1686 - acc: 0.7570 - val_loss: 0.5152 - val_mean_squared_error: 0.1707 - val_acc: 0.7554
E

53176/53176 [==============================] - 2s - loss: 0.4718 - mean_squared_error: 0.1543 - acc: 0.7788 - val_loss: 0.5350 - val_mean_squared_error: 0.1783 - val_acc: 0.7414
Epoch 44/100
53176/53176 [==============================] - 2s - loss: 0.4714 - mean_squared_error: 0.1543 - acc: 0.7777 - val_loss: 0.5327 - val_mean_squared_error: 0.1769 - val_acc: 0.7434
Epoch 45/100
53176/53176 [==============================] - 2s - loss: 0.4704 - mean_squared_error: 0.1538 - acc: 0.7794 - val_loss: 0.5341 - val_mean_squared_error: 0.1780 - val_acc: 0.7426
Epoch 46/100
53176/53176 [==============================] - 2s - loss: 0.4695 - mean_squared_error: 0.1536 - acc: 0.7802 - val_loss: 0.5352 - val_mean_squared_error: 0.1776 - val_acc: 0.7453
Epoch 47/100
53176/53176 [==============================] - 2s - loss: 0.4689 - mean_squared_error: 0.1533 - acc: 0.7801 - val_loss: 0.5365 - val_mean_squared_error: 0.1783 - val_acc: 0.7429
Epoch 48/100
53176/53176 [==============================] 

53176/53176 [==============================] - 2s - loss: 0.4466 - mean_squared_error: 0.1450 - acc: 0.7929 - val_loss: 0.5613 - val_mean_squared_error: 0.1844 - val_acc: 0.7368
Epoch 86/100
53176/53176 [==============================] - 2s - loss: 0.4451 - mean_squared_error: 0.1444 - acc: 0.7940 - val_loss: 0.5598 - val_mean_squared_error: 0.1843 - val_acc: 0.7349
Epoch 87/100
53176/53176 [==============================] - 2s - loss: 0.4444 - mean_squared_error: 0.1443 - acc: 0.7948 - val_loss: 0.5613 - val_mean_squared_error: 0.1858 - val_acc: 0.7316
Epoch 88/100
53176/53176 [==============================] - 2s - loss: 0.4442 - mean_squared_error: 0.1441 - acc: 0.7935 - val_loss: 0.5663 - val_mean_squared_error: 0.1847 - val_acc: 0.7382
Epoch 89/100
53176/53176 [==============================] - 2s - loss: 0.4435 - mean_squared_error: 0.1439 - acc: 0.7933 - val_loss: 0.5739 - val_mean_squared_error: 0.1873 - val_acc: 0.7319
Epoch 90/100
53176/53176 [==============================] 

In [45]:
X_eval, ids = process_data(test_path, True) 

outputs = l1.predict(X_eval)
outputs = [x[1] for x in outputs[0]]

submission = ['ID,PredictedProb']

for prediction, id in zip(outputs, ids):
    submission.append('{0},{1}'.format(id,prediction))

submission = '\n'.join(submission)

with open('/Users/ngochuyenluu/Downloads/BNP/sample_submission.csv', 'w') as outfile:
    outfile.write(submission)